# Import libraries

In [1]:
import pandas as pd
import numpy as np
import os
import config
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_predict
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import balanced_accuracy_score, accuracy_score
from sklearn.metrics import confusion_matrix
from tqdm import tqdm

# Load data

In [2]:
X = pd.read_csv(os.path.join(config.CLEAN_DIR, "taxonomic_features.csv"), index_col=[0, 1])
y = pd.read_csv(os.path.join(config.CLEAN_DIR, "is_healthy.csv"), index_col=[0, 1])
display(X.head())
display(y.head())

k__Archaea  k__Archaea|p__Euryarchaeota  \
Study_ID Sample Accession                                            
GMHI-10  SAMN03283239        0.000000                     0.000000   
         SAMN03283266        0.000000                     0.000000   
         SAMN03283281        0.009764                     0.009764   
         SAMN03283245        0.106067                     0.106067   
         SAMN03283294        0.000000                     0.000000   

                           k__Archaea|p__Euryarchaeota|c__Methanobacteria  \
Study_ID Sample Accession                                                   
GMHI-10  SAMN03283239                                            0.000000   
         SAMN03283266                                            0.000000   
         SAMN03283281                                            0.009764   
         SAMN03283245                                            0.106067   
         SAMN03283294                                            0.000000   

                           k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales  \
Study_ID Sample Accession                                                                         
GMHI-10  SAMN03283239                                               0.000000                      
         SAMN03283266                                               0.000000                      
         SAMN03283281                                               0.009764                      
         SAMN03283245                                               0.106067                      
         SAMN03283294                                               0.000000                      

                           k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales|f__Methanobacteriaceae  \
Study_ID Sample Accession                                                                                                
GMHI-10  SAMN03283239                                               0.000000                                             
         SAMN03283266                                               0.000000                                             
         SAMN03283281                                               0.009764                                             
         SAMN03283245                                               0.106067                                             
         SAMN03283294                                               0.000000                                             

                           k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales|f__Methanobacteriaceae|g__Methanobrevibacter  \
Study_ID Sample Accession                                                                                                                      
GMHI-10  SAMN03283239                                               0.000000                                                                   
         SAMN03283266                                               0.000000                                                                   
         SAMN03283281                                               0.009764                                                                   
         SAMN03283245                                               0.072453                                                                   
         SAMN03283294                                               0.000000                                                                   

                           k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales|f__Methanobacteriaceae|g__Methanobrevibacter|s__Methanobrevibacter_smithii  \
Study_ID Sample Accession                                                                                                                                                    
GMHI-10  SAMN03283239                                               0.000000              

PHENOTYPE:Healthy_Nonhealthy
Study_ID Sample Accession                              
GMHI-10  SAMN03283239                              True
         SAMN03283266                              True
         SAMN03283281                              True
         SAMN03283245                              True
         SAMN03283294                              True

# Generate predictions in 10-fold cross validation! (repeated 10 times)

In [3]:
num_repeats = 10
n_splits = 10
kfold = KFold(n_splits=n_splits, shuffle=True)

logit_prediction_list = []

for i in tqdm(range(num_repeats)):
    # instantiate the logistic regression classifier (gmhi2)
    gmhi2 = LogisticRegression(random_state=42, penalty="l1", solver="liblinear", C=config.REGULARIZATION, class_weight="balanced")

    # generate logit predictions in 10 fold cross validation
    GMHI2_scores_cv = cross_val_predict(gmhi2, X > config.PRESENCE_CUTOFF, y.values.flatten(), method="decision_function",
        cv=kfold, verbose=0, n_jobs=-1
    )

    GMHI2_scores_cv = pd.DataFrame(GMHI2_scores_cv, index=y.index, columns=["GMHI2_cv"])
    logit_prediction_list.append(GMHI2_scores_cv)

100%|███████████████████████████████████████████████████████████████| 10/10 [00:24<00:00,  2.41s/it]


In [9]:
def evaluate_performance(cutoff):
    balanced_accuracies = [balanced_accuracy_score(y[(abs(lo) >= cutoff).values], lo[(abs(lo) >= cutoff).values] > 0) for lo in logit_prediction_list]
    mean_acc = np.mean(balanced_accuracies)
    print("Mean acc:", mean_acc)
    
    mat_sum = np.zeros([2 , 2])

    for lo in logit_prediction_list:
        idx = (abs(lo) >= cutoff).values
        y_curr = y[idx]
        lo_curr = lo[idx]
        mat = confusion_matrix(y_curr, lo_curr > 0)
        mat_sum += mat
    tn, fp, fn, tp = mat_sum.ravel()
    df = pd.DataFrame(mat_sum, columns=["Predicted Nonhealthy", "Predicted Healthy"], index=["Actual Nonhealthy", "Actual healthy"])
    display(df)
    print("Total samples evaluated:", mat_sum.sum())
    print("Percentage of possible:", mat_sum.sum() / X.shape[0] * 10)

In [10]:
evaluate_performance(0)

Mean acc: 0.7776192350834326


,Predicted Nonhealthy,Predicted Healthy
Actual Nonhealthy,24449.0,6671.0
Actual healthy,13264.0,44306.0


Total samples evaluated: 88690.0
Percentage of possible: 100.0


In [11]:
evaluate_performance(0.1)

Mean acc: 0.7912313023773946


,Predicted Nonhealthy,Predicted Healthy
Actual Nonhealthy,23632.0,5902.0
Actual healthy,11958.0,42970.0


Total samples evaluated: 84462.0
Percentage of possible: 95.23283346487766


In [12]:
evaluate_performance(1)

Mean acc: 0.8933505374266286


,Predicted Nonhealthy,Predicted Healthy
Actual Nonhealthy,13998.0,1554.0
Actual healthy,3688.0,28841.0


Total samples evaluated: 48081.0
Percentage of possible: 54.21242530161236
